In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
alphavantage_api_key=os.environ.get("ALPHAVANTAGE_API_KEY")


In [3]:
from langchain_community.utilities.alpha_vantage import AlphaVantageAPIWrapper

In [4]:
alpha_vantage = AlphaVantageAPIWrapper()
alpha_vantage._get_exchange_rate("USD", "JPY")

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD',
  '2. From_Currency Name': 'United States Dollar',
  '3. To_Currency Code': 'JPY',
  '4. To_Currency Name': 'Japanese Yen',
  '5. Exchange Rate': '147.57900000',
  '6. Last Refreshed': '2024-08-18 10:50:04',
  '7. Time Zone': 'UTC',
  '8. Bid Price': '147.57330000',
  '9. Ask Price': '147.58030000'}}

In [14]:
def alphavantage_get_exchange_rate(currency1: str, currency2: str):
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage._get_exchange_rate(currency1, currency2))

In [15]:
alphavantage_get_exchange_rate("USD","JPY")

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD',
  '2. From_Currency Name': 'United States Dollar',
  '3. To_Currency Code': 'JPY',
  '4. To_Currency Name': 'Japanese Yen',
  '5. Exchange Rate': '147.57900000',
  '6. Last Refreshed': '2024-08-18 11:05:34',
  '7. Time Zone': 'UTC',
  '8. Bid Price': '147.57330000',
  '9. Ask Price': '147.58030000'}}

In [33]:
from langchain_core.tools import tool

@tool
def alphavantage_get_exchange_rate(currency1: str, currency2: str):
    """Get exchange Rates using Alpha Vantage API."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage._get_exchange_rate(currency1, currency2))

@tool
def alphavantage_get_time_series_daily(symbol: str):
    """return the date, daily open, daily high, daily low, daily close, and daily volume of the global equity specified, covering the 100 latest data points."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage._get_time_series_daily(symbol))

@tool
def alphavantage_get_time_series_weekly(symbol: str):
    """Return the last trading day of the week, weekly open, weekly high, weekly low, weekly close, and weekly volume of the global equity specified, covering 20+ years of historical data."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage._get_time_series_weekly(symbol))

@tool
def alphavantage_get_quote_endpoint(symbol: str):
    """ lightweight alternative to the time series APIs and returns the latest price and volume info for the specified symbol."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage._get_quote_endpoint(symbol))

@tool
def alphavantage_search_symbols(symbol: str):
    """returns a list of symbols and the matching company information based on the text entered."""
    alpha_vantage = AlphaVantageAPIWrapper()
    return (alpha_vantage.search_symbols(symbol))

In [34]:
from langchain_openai import ChatOpenAI

tools = [alphavantage_get_exchange_rate,alphavantage_get_time_series_daily,alphavantage_get_time_series_weekly,alphavantage_get_quote_endpoint,alphavantage_search_symbols]

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(tools)

In [35]:
from langchain import hub
from langchain.agents import create_tool_calling_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

agent = create_tool_calling_agent(llm, tools, prompt)

c:\Code\Github\LangChain-Basics\venv\lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [36]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [40]:
# query = "What is the current exchange rate from USD to JPY?"

# query = "get time series weekly data for Tesla?"

# query = "get daily  data for Tesla?"

# query = "get quote for Tesla?"

query = "What is symbol for microsoft and quote for Tesla?"


# response = agent_executor.invoke(query)

agent_executor.invoke({"input": query})




> Entering new AgentExecutor chain...

Invoking: `alphavantage_search_symbols` with `{'symbol': 'Microsoft'}`


{'bestMatches': [{'1. symbol': 'MSF0.FRK', '2. name': 'MICROSOFT CORP. CDR', '3. type': 'Equity', '4. region': 'Frankfurt', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '0.6429'}, {'1. symbol': 'MSFT', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.6154'}, {'1. symbol': '0QYP.LON', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United Kingdom', '5. marketOpen': '08:00', '6. marketClose': '16:30', '7. timezone': 'UTC+01', '8. currency': 'USD', '9. matchScore': '0.6000'}, {'1. symbol': 'MSF.DEX', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'XETRA', '5. marketOpen': '08:00', '6. marketClose': '20:00'

{'input': 'What is symbol for microsoft and quote for Tesla?',
 'output': 'The symbol for Microsoft is **MSFT** (Microsoft Corporation).\n\nAs for Tesla (symbol: **TSLA**), here is the latest quote:\n\n- **Open:** $211.15\n- **High:** $219.80\n- **Low:** $210.80\n- **Price:** $216.12\n- **Volume:** 88,765,122\n- **Latest Trading Day:** August 16, 2024\n- **Previous Close:** $214.14\n- **Change:** $1.98\n- **Change Percent:** 0.92%'}